In [1]:
# https://transparencyreport.google.com/
# https://transparencyreport.google.com/safe-browsing/search?url=www.example.com
# https://cnsviewer.corp.google.com/placer/prod/home/trustrank-team/badurls/badurls_phishing_landing_url

import csv
import datetime
from faker import Faker
import pandas as pd
import pandas_gbq

from google.cloud import webrisk_v1
from google.cloud.webrisk_v1 import SearchUrisResponse
    
project_id = "genai-embeddings"
table_id = 'appgate.webrisk_lookups'

def search_uri(
    uri: str, threat_type
) -> SearchUrisResponse:

    webrisk_client = webrisk_v1.WebRiskServiceClient()

    request = webrisk_v1.SearchUrisRequest()
    request.threat_types = threat_type
    request.uri = uri

    response = webrisk_client.search_uris(request)
    if response.threat.threat_types:
        print(f"The URI has the following threat: {response}")
        
    else:
        print("The URL is safe!")
    return response


fake = Faker()
    
with open('./urls.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    urls = [row["url"] for row in reader]


#"my_dates": pandas.date_range("now", periods=3),

df1 = pd.DataFrame(columns=['customer','lat','lng','latlong','place','country','timezone','url','operation_type','operation_status','threat','threat_types','scanned_time'])



counter=0

company=fake.company()
fakedata=fake.location_on_land()
lat = fakedata[0]
lng = fakedata[1]
latlng=lat + ',' + lng
place=fakedata[2]
country=fakedata[3]
timez=fakedata[4]
    
for i in urls:
    print('#################################################')
    print(i)
    
    counter+=1
    
    now = datetime.datetime.now()
    joined_threats=''
    treath_found=False

    operation_type='Lookup'
    operation_status='SUCCEEDED'


    ## Generate fake data
    
    if counter > 5:
        company=fake.company()
        fakedata=fake.location_on_land()
        lat = fakedata[0]
        lng = fakedata[1]
        latlng=lat + ',' + lng
        place=fakedata[2]
        country=fakedata[3]
        timez=fakedata[4]
        counter=0

    
    uri=i
    threat_list=[webrisk_v1.ThreatType.MALWARE,
                 webrisk_v1.ThreatType.SOCIAL_ENGINEERING,
                 webrisk_v1.ThreatType.UNWANTED_SOFTWARE,
                 webrisk_v1.ThreatType.SOCIAL_ENGINEERING_EXTENDED_COVERAGE]
    response = search_uri(
            uri,
            threat_list,
    )
    
    res_list=list(response.threat.threat_types)

    for x in res_list:
        #print(str(x))
        joined_threats= str(x) + ';'  + joined_threats 

    if len(joined_threats) > 0:
        treath_found=True
        
    print(joined_threats)
    
    # Adding a lookup row type
    new_row = {
            "customer": company,
            "lat": lat,
            "lng": lng,
            "latlong": latlng,
            "place": place,
            "country": country,
            "timezone": timez,
            "url": uri,
            "operation_type": operation_type,
            "operation_status": operation_status ,
            "threat": treath_found,
            "threat_types": joined_threats,        
            "scanned_time": now
        }
    df1.loc[len(df1)] = new_row


    # Adding a submission row type
    if treath_found is True:
        operation_type='Submit'
        if counter > 3: 
            operation_status='SUCCEEDED'
        else:    
            operation_status='CLOSED'

        new_row = {
                "customer": company,
                "lat": lat,
                "lng": lng,
                "latlong": latlng,
                "place": place,
                "country": country,
                "timezone": timez,
                "url": uri,
                "operation_type": operation_type,
                "operation_status": operation_status ,
                "threat": treath_found,
                "threat_types": joined_threats,        
                "scanned_time": now
            }
        df1.loc[len(df1)] = new_row
    
pandas_gbq.to_gbq(df1, table_id, project_id=project_id, if_exists='replace')  # replace to replace table; append to append to a table
    

#################################################
http://testsafebrowsing.appspot.com/s/malware.html
The URI has the following threat: threat {
  threat_types: MALWARE
  expire_time {
    seconds: 1701280183
    nanos: 730427404
  }
}

ThreatType.MALWARE;
#################################################
http://testsafebrowsing.appspot.com/malware.html
The URL is safe!

#################################################
http://www.google.com
The URL is safe!

#################################################
http://example.com
The URL is safe!

#################################################
https://ngif.newcastle.ac.uk/download-all/Lysimeter%203/Rain%20-%20Monthly%20Total
The URL is safe!

#################################################
https://ngif.newcastle.ac.uk/download-all/Lysimeter%204/A_5TE_soilmoisture_3_75cm_road
The URL is safe!

#################################################
https://ngif.newcastle.ac.uk/download-all/Lysimeter%204/A_5TE_temperature_4_20cm_in
The URL is

100%|██████████| 1/1 [00:00<00:00, 13066.37it/s]
